# Course Project - NBA data analysis (Due: 20 Dec 2021)

Group BD (AKA basketball data analysis)

## Task-1
1. [40%] Your boss would like to know the top-3 players of the best [cost–performance ratio](https://en.wikipedia.org/wiki/Price%E2%80%93performance_ratio) in 2020/2021 season, where the cost and the performance can be calculated by
    - PERFORMANCE: [Efficiency equation](https://en.wikipedia.org/wiki/Efficiency_(basketball)#:~:text=The%20NBA%20publishes%20online%20all%20of%20the%20basic,FG%20%E2%88%92%20Missed%20FT%20-%20TO%29%20%2F%20GP) based on the statistics in the **(3) Kaggle 2020/2021 season data**.
        - You could replace this equation if there is any better option.
    - COST: Salary in talbe "Player_Salary" of **(1) Kaggle basketball dataset**.
    - You are welcome to develop any better eqaution(s) to estimate the CP ratio to rank the top-3 players.


In [1]:
import pandas as pd
import numpy as np
import sqlite3

"""
Efficiency equation：
(PTS + REB + AST + STL + BLK − Missed FG − Missed FT - TO) / GP
PTS: Points 得分
REB, OREB, DREB: (Rebound) 篮板, 进攻篮板, 防守篮板
AST： (Assist缩写) 助攻
STL: Steals 抢断、断球
BLK: Blocks 盖帽
FG: Field Goals 投球命中（次数）/投篮(总称)，包括两分、三分球的投篮
FT：Free Throws 罚球
TOV（或TO）: Turnovers 失误次数
GM, GP; GS: games played; games started
"""


con = sqlite3.connect('datasets/basketball.sqlite')
cur = con.cursor()

df = pd.read_csv('datasets/nba2021_per_game.csv')

cps = {}
rank = []


def Efficiency(r):
    return r['PTS'] + r['ORB'] + r['DRB'] + r['AST'] + r['STL'] + r['BLK'] - (r['FGA'] - r['FG']) \
           - (r['FTA'] - r['FT']) - r['TOV']


def get_salary(name):
    salarys = pd.read_sql_query("SELECT value FROM Player_Salary WHERE namePlayer = ?", con, params=(name,))
    return np.average(salarys) / 1000000 if len(salarys) != 0 else -1


for i in range(len(df)):
    record = df.iloc[i]
    EFF = Efficiency(record)
    salary = get_salary(record['Player'])
    if salary == -1:
        continue
    name = EFF / salary
    if record['Player'] not in cps.keys():
        cps[record['Player']] = []
    cps[record['Player']].append(name)

for name in cps.keys():
    if len(cps[name]) > 1:
        cps[name] = np.average(cps[name])
    rank.append([name, cps[name]])
list.sort(rank, key=lambda x: x[1], reverse=True)
print('the top-3 players are:')
for i in range(3):
    print(*rank[i])

the top-3 players are:
Chimezie Metu [94.0]
Mason Jones [81.26381484852426]
Yogi Ferrell [76.57795635957405]
